# 🎯 Advanced Grasp Detection v5.1 (Full Calibration)

**ระบบ Pick-and-Place พร้อม Calibration Tools ครบชุด**

### ✨ Features
| Feature | Description |
|---------|-------------|
| **📏 Depth Calibration** | Calibrate ค่า DEPTH_Z_SCALE |
| **📐 Pixel/mm Calibration** | Calibrate ค่า PIXELS_PER_MM |
| **🦾 Gripper Calibration** | ทดสอบ gripper width mapping |
| **🎯 Full Pick-and-Place** | ระบบหยิบวัตถุอัตโนมัติ |

## 1️⃣ Imports & Setup

In [1]:
import sys
import cv2
import numpy as np
import time
import socket
import serial
import torch

sys.path.append('Depth-Anything-V2')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print("✓ Imports loaded")

PyTorch: 2.9.1+cpu
CUDA: False
✓ Imports loaded


## 2️⃣ Configuration (⚠️ แก้ไขค่าหลัง Calibration)

In [2]:
# =================================================================
# Hardware Configuration
# =================================================================
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

# =================================================================
# Homography Matrix (Camera → Robot)
# =================================================================
HOMOGRAPHY_MATRIX = np.array([
    [0.005703976266962427, -0.3265299161278153, 88.58634169557483],
    [-0.47704058225560797, 0.015355046930804153, 172.0941543570439],
    [-0.00029949919510557677, 0.00018728182448344945, 1.0],
], dtype=np.float32)

# =================================================================
# ⚠️ CALIBRATION VALUES - แก้ไขหลังจาก Calibrate!
# =================================================================
PIXELS_PER_MM = 2.2167      # 📐 ได้จาก Pixel/mm Calibration
DEPTH_Z_SCALE = 57.428993     # 📏 ได้จาก Depth Calibration

# =================================================================
# Z Heights
# =================================================================
Z_FLOOR = -64
Z_SAFE = -40
Z_APPROACH = -55

# =================================================================
# Drop Position
# =================================================================
DROP_POS = (-253.07, 115.17, -17.07, -62.78)

# =================================================================
# Gripper Configuration
# =================================================================
GRIPPER_SERVO_OPEN_ANGLE = 22
GRIPPER_SERVO_CLOSE_ANGLE = 96
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_MIN_WIDTH_MM = 0
GRIPPER_OPEN_MARGIN_MM = 15
GRIPPER_GRIP_MARGIN_MM = 3

# =================================================================
# Detection Settings
# =================================================================
MIN_OBJECT_AREA = 1000
THRESHOLD_VALUE = 30

# =================================================================
# Depth Model
# =================================================================
DEPTH_MODEL_PATH = 'Depth-Anything-V2/checkpoints/depth_anything_v2_vits.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("✓ Configuration loaded")
print(f"  PIXELS_PER_MM: {PIXELS_PER_MM}")
print(f"  DEPTH_Z_SCALE: {DEPTH_Z_SCALE}")

✓ Configuration loaded
  PIXELS_PER_MM: 2.2167
  DEPTH_Z_SCALE: 57.428993


## 3️⃣ Load Depth Model

In [3]:
from depth_anything_v2.dpt import DepthAnythingV2

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
}

print("Loading DepthAnything V2...")
depth_model = DepthAnythingV2(**model_configs['vits'])
depth_model.load_state_dict(torch.load(DEPTH_MODEL_PATH, map_location='cpu'))
depth_model = depth_model.to(DEVICE).eval()
print(f"✅ Depth model loaded on {DEVICE}")

xFormers not available
xFormers not available


Loading DepthAnything V2...
✅ Depth model loaded on cpu


## 4️⃣ Classes Definition

In [4]:
class SmartGripperController:
    # =====================================================
    # 📊 CALIBRATION DATA (วัดจริง 7 Dec 2025)
    # =====================================================
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.current_angle = GRIPPER_SERVO_OPEN_ANGLE
        self.target_object_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            self.serial.reset_input_buffer()
            print(f"✅ Gripper connected on {self.port}")
            return True
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False
    
    def disconnect(self):
        if self.serial:
            self.serial.close()
    
    def send_command(self, cmd):
        if not self.serial:
            return None
        self.serial.reset_input_buffer()
        self.serial.write((cmd + '\n').encode())
        time.sleep(0.3)
        response = ""
        while self.serial.in_waiting:
            response += self.serial.readline().decode().strip() + "\n"
        return response.strip()
    
    def mm_to_angle(self, width_mm):
        """แปลง width (mm) → angle (°) ด้วย Interpolation"""
        width = max(0.0, min(54.0, width_mm))
        angle = np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])
        return int(round(angle))
    
    def open_for_object(self, object_width_mm):
        self.target_object_width = object_width_mm
        open_width = min(54.0, object_width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_width)
        print(f"🦾 Opening for {object_width_mm:.1f}mm → {open_width:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
        self.current_angle = angle
    
    def grip_object(self, object_width_mm):
        grip_width = max(0.0, object_width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_width)
        angle = min(96, angle + 3)
        print(f"🦾 Gripping {object_width_mm:.1f}mm → {grip_width:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
        self.current_angle = angle
    
    def release(self):
        release_width = (self.target_object_width + 10) if self.target_object_width else 54.0
        release_width = min(54.0, release_width)
        angle = self.mm_to_angle(release_width)
        print(f"🦾 Releasing → {release_width:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
        self.current_angle = angle
        self.target_object_width = None
    
    def full_open(self):
        angle = self.mm_to_angle(54.0)
        self.send_command(f'G{angle}')
        self.current_angle = angle

class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock_dashboard = None
        self.homography_matrix = homography_matrix
        
    def connect(self, ip):
        try:
            self.sock_dashboard = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock_dashboard.settimeout(5)
            self.sock_dashboard.connect((ip, self.dashboard_port))
            print("Clearing Errors...")
            self.send_command("ClearError()")
            time.sleep(0.5)
            print("Enabling Robot...")
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Connection Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock_dashboard:
            try:
                self.sock_dashboard.send((cmd + "\n").encode("utf-8"))
                return self.sock_dashboard.recv(1024).decode("utf-8")
            except Exception as e:
                print(f"Command Error: {e}")
        return None

    def home(self):
        """🏠 กลับตำแหน่ง HOME"""
        print("🤖 Moving to HOME...")
        #self.send_command("JointMovJ(0.19,-12.21,38.29,59.21)")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)
        print("✅ HOME")

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait_time=3):
        self.move_to(x, y, z, r)
        time.sleep(wait_time)

    def pixel_to_robot(self, u, v):
        if self.homography_matrix is None:
            return None, None
        point = np.array([u, v, 1], dtype=np.float32)
        result = np.dot(self.homography_matrix, point)
        return result[0] / result[2], result[1] / result[2]
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        """🔄 Move using joint angles"""
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait_time=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait_time)

class AdvancedObjectDetector:
    def __init__(self):
        self.background = None
        self.background_gray = None
        
    def set_background(self, frame):
        self.background = frame.copy()
        self.background_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.background_gray = cv2.GaussianBlur(self.background_gray, (5, 5), 0)
        print("✅ Background captured!")
    
    def detect_objects(self, frame, threshold=40, min_area=2000):
        if self.background_gray is None:
            return []
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        diff = cv2.absdiff(self.background_gray, gray)
        _, thresh = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        thresh = cv2.erode(thresh, kernel, iterations=2)
        thresh = cv2.dilate(thresh, kernel, iterations=2)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objects = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > min_area:
                hull = cv2.convexHull(contour)
                x, y, w, h = cv2.boundingRect(hull)
                cx, cy = x + w // 2, y + h // 2
                rect = cv2.minAreaRect(hull)
                objects.append({
                    'bbox': (x, y, w, h), 'center': (cx, cy), 'contour': hull,
                    'rect': rect, 'rect_size': rect[1], 'rect_angle': rect[2], 'area': area
                })
        objects.sort(key=lambda o: o['area'], reverse=True)
        return objects
    
    def get_grip_params(self, obj, pixels_per_mm):
        rect_w, rect_h = obj['rect_size']
        angle = obj['rect_angle']
        if rect_w < rect_h:
            grip_width_px = rect_w
            grip_angle = angle + 90
        else:
            grip_width_px = rect_h
            grip_angle = angle
        while grip_angle > 90: grip_angle -= 180
        while grip_angle < -90: grip_angle += 180
        grip_width_mm = grip_width_px / pixels_per_mm
        robot_r = -grip_angle
        return grip_width_mm, grip_angle, robot_r

class DepthBasedZEstimator:
    def __init__(self, model, device='cpu'):
        self.model = model
        self.device = device
        self.floor_depth = None
        
    def estimate_depth(self, frame):
        return self.model.infer_image(frame)
    
    def calibrate_floor(self, frame):
        depth = self.estimate_depth(frame)
        h, w = depth.shape
        center_region = depth[h//3:2*h//3, w//3:2*w//3]
        self.floor_depth = np.median(center_region)
        print(f"✅ Floor depth: {self.floor_depth:.4f}")
        return self.floor_depth
    
    def get_object_height_mm(self, depth_map, obj, scale):
        if self.floor_depth is None:
            return 0
        x, y, w, h = obj['bbox']
        obj_region = depth_map[y:y+h, x:x+w]
        if obj_region.size == 0:
            return 0
        obj_depth = np.median(obj_region)
        depth_diff = obj_depth - self.floor_depth
        return max(0, depth_diff * scale)
    
    def calculate_grasp_z(self, height_mm):
        z_grasp = Z_FLOOR + (height_mm * 0.5)
        return max(Z_FLOOR, min(Z_SAFE, z_grasp))

print("✓ All classes loaded")

✓ All classes loaded


## 5️⃣ Initialize Components

In [5]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)
detector = AdvancedObjectDetector()
depth_estimator = DepthBasedZEstimator(depth_model, device=DEVICE)
print("✓ All components initialized")

✓ All components initialized


In [6]:
gripper.connect()

✅ Gripper connected on COM9


True

In [7]:
robot.connect(ROBOT_IP)

Clearing Errors...
Enabling Robot...
✅ Robot connected!


True

---
# 🔧 CALIBRATION SECTION
---

## 📐 Calibration 1: PIXELS_PER_MM

**วิธีใช้:**
1. วางวัตถุที่รู้ขนาด (เช่น ไม้บรรทัด, กล่อง) บนพื้นที่ทำงาน
2. Run cell แล้วลากเส้นจากปลายหนึ่งไปอีกปลาย
3. ใส่ขนาดจริง (mm) ที่ terminal
4. คัดลอกค่า PIXELS_PER_MM ไปใส่ใน Configuration

In [ ]:
# =================================================================
# 📐 PIXELS_PER_MM CALIBRATION
# =================================================================
print("="*60)
print("📐 PIXELS_PER_MM CALIBRATION")
print("="*60)
print("1. วางวัตถุที่รู้ขนาดจริง (เช่น ไม้บรรทัด)")
print("2. คลิกลากจากจุดเริ่ม → จุดสิ้นสุด")
print("3. กด ENTER แล้วใส่ขนาดจริง (mm)")
print("Q = Quit")
print("="*60)

drawing = False
start_point = None
end_point = None
pixel_distance = 0

def pixel_calib_callback(event, x, y, flags, param):
    global drawing, start_point, end_point, pixel_distance
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        end_point = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        end_point = (x, y)
        pixel_distance = np.sqrt((end_point[0]-start_point[0])**2 + (end_point[1]-start_point[1])**2)
        print(f"📏 Pixel distance: {pixel_distance:.1f} pixels")

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pixel Calibration')
cv2.setMouseCallback('Pixel Calibration', pixel_calib_callback)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    if start_point and end_point:
        cv2.line(frame, start_point, end_point, (0, 255, 0), 2)
        cv2.circle(frame, start_point, 5, (0, 0, 255), -1)
        cv2.circle(frame, end_point, 5, (0, 0, 255), -1)
        cv2.putText(frame, f"{pixel_distance:.1f}px", 
                   ((start_point[0]+end_point[0])//2, (start_point[1]+end_point[1])//2 - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    cv2.putText(frame, "Drag line on known-size object | ENTER=Calculate | Q=Quit",
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.imshow('Pixel Calibration', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 13 and pixel_distance > 0:  # Enter key
        try:
            real_mm = float(input("\n📏 ใส่ขนาดจริง (mm): "))
            calculated_ppm = pixel_distance / real_mm
            print("\n" + "="*60)
            print(f"✅ PIXELS_PER_MM = {calculated_ppm:.4f}")
            print("="*60)
            print("คัดลอกค่านี้ไปใส่ใน Configuration cell!")
        except:
            print("❌ Invalid input")

cap.release()
cv2.destroyAllWindows()

## 📏 Calibration 2: DEPTH_Z_SCALE

**วิธีใช้:**
1. ใส่ความสูงจริงของวัตถุที่ใช้ calibrate (KNOWN_OBJECT_HEIGHT_MM)
2. กด C เพื่อ calibrate พื้น (เอาวัตถุออก)
3. วางวัตถุ แล้วคลิกที่วัตถุ
4. คัดลอกค่า DEPTH_Z_SCALE ไปใส่ใน Configuration

In [ ]:
# =================================================================
# 📏 DEPTH_Z_SCALE CALIBRATION
# =================================================================

# ⚠️ ใส่ความสูงจริงของวัตถุที่ใช้ calibrate (mm)
KNOWN_OBJECT_HEIGHT_MM = 11.5  # 🔴 แก้ค่านี้!

print("="*60)
print("📏 DEPTH_Z_SCALE CALIBRATION")
print("="*60)
print(f"⚠️ เตรียมวัตถุความสูง: {KNOWN_OBJECT_HEIGHT_MM} mm")
print("C = Calibrate พื้น (เอาวัตถุออก)")
print("Click = เลือกวัตถุเพื่อ calibrate")
print("Q = Quit")
print("="*60)

calibration_object = None
calibrated_scale = None
detected_objects = []
current_depth_map = None

def depth_calib_callback(event, x, y, flags, param):
    global calibration_object, calibrated_scale
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in detected_objects:
            bx, by, bw, bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                if current_depth_map is not None and depth_estimator.floor_depth is not None:
                    ox, oy, ow, oh = obj['bbox']
                    obj_region = current_depth_map[oy:oy+oh, ox:ox+ow]
                    obj_depth = np.median(obj_region)
                    raw_diff = obj_depth - depth_estimator.floor_depth
                    
                    if raw_diff > 0:
                        calibrated_scale = KNOWN_OBJECT_HEIGHT_MM / raw_diff
                        print("\n" + "="*60)
                        print(f"   Floor depth: {depth_estimator.floor_depth:.4f}")
                        print(f"   Object depth: {obj_depth:.4f}")
                        print(f"   Raw diff: {raw_diff:.4f}")
                        print("="*60)
                        print(f"✅ DEPTH_Z_SCALE = {calibrated_scale:.2f}")
                        print("="*60)
                    else:
                        print("❌ Depth diff <= 0, calibrate พื้นใหม่")
                else:
                    print("❌ กด C เพื่อ calibrate พื้นก่อน")
                break

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Depth Calibration')
cv2.setMouseCallback('Depth Calibration', depth_calib_callback)

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame_count += 1
    if frame_count % 5 == 0:
        current_depth_map = depth_estimator.estimate_depth(frame)
    
    detected_objects = detector.detect_objects(frame, THRESHOLD_VALUE, MIN_OBJECT_AREA)
    
    for obj in detected_objects:
        x, y, w, h = obj['bbox']
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    floor_status = f"Floor: {depth_estimator.floor_depth:.3f}" if depth_estimator.floor_depth else "Floor: NOT SET"
    cv2.putText(frame, f"{floor_status} | C=Calibrate Floor | Click=Select | Q=Quit",
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    if calibrated_scale:
        cv2.putText(frame, f"SCALE = {calibrated_scale:.2f}", (10, 460),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    cv2.imshow('Depth Calibration', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        print("Calibrating floor...")
        depth_estimator.calibrate_floor(frame)
        detector.set_background(frame)

cap.release()
cv2.destroyAllWindows()

if calibrated_scale:
    print("\n📋 Copy this: DEPTH_Z_SCALE =", calibrated_scale)

## 🦾 Calibration 3: Gripper Width Test

**วิธีใช้:**
1. ใส่ค่าความกว้าง (mm) เพื่อทดสอบ
2. วัดความกว้าง gripper จริงด้วยไม้บรรทัด
3. ถ้าไม่ตรง ให้ปรับค่า GRIPPER_MAX_WIDTH_MM และ GRIPPER_MIN_WIDTH_MM

In [ ]:
gripper.open_for_object(23)

In [ ]:
gripper.open_for_object(0)

In [ ]:
# ทดสอบ width โดยตรง
width = 23
angle = gripper.mm_to_angle(width)
gripper.send_command(f'G{angle}')
time.sleep(5)
print(f"Target: {width}mm, Angle: {angle}°")
# แล้ววัดจริงดู

In [ ]:
# =================================================================
# 🔧 GRIPPER CALIBRATION DATA COLLECTION
# =================================================================
print("="*60)
print("🔧 GRIPPER CALIBRATION - เก็บข้อมูล")
print("="*60)
print("ผมจะส่ง angle ไปที่ servo")
print("คุณแค่วัดความกว้าง gripper แล้วพิมพ์ค่า (mm)")
print("="*60)

calibration_data = []
test_angles = [22, 30, 40, 50, 60, 70, 80, 90, 96]

for angle in test_angles:
    gripper.send_command(f'G{angle}')
    time.sleep(5)  # รอให้ servo ขยับเสร็จ
    
    measured = input(f"Angle {angle}° → วัดความกว้างได้เท่าไร (mm)? : ")
    try:
        width = float(measured)
        calibration_data.append((angle, width))
        print(f"   ✅ บันทึก: {angle}° = {width}mm")
    except:
        print(f"   ❌ ข้ามค่านี้")

print("\n" + "="*60)
print("📊 ข้อมูล Calibration:")
print("="*60)
for angle, width in calibration_data:
    print(f"   {angle}° → {width}mm")
print("="*60)

# เปิด gripper กลับ
gripper.send_command('G22')

In [ ]:
# grip calibrate เก่า =================================================================
# 🦾 GRIPPER WIDTH CALIBRATION TEST
# =================================================================
print("="*60)
print("🦾 GRIPPER WIDTH TEST")
print("="*60)
print("ใส่ค่าความกว้าง (mm) แล้ววัดจริงเทียบกับค่า")
print(f"  22° = {GRIPPER_MAX_WIDTH_MM}mm (เปิดสุด)")
print(f"  96° = {GRIPPER_MIN_WIDTH_MM}mm (ปิดสุด)")
print("Commands: [number]=mm, o=open, c=close, q=quit")
print("="*60)

test_values = [74, 60, 50, 40, 30, 20, 10, 5]

while True:
    cmd = input("\n🦾 Width (mm) or command: ").strip().lower()
    if cmd == 'q':
        break
    elif cmd == 'o':
        gripper.full_open()
        print(f"Opened to {GRIPPER_MAX_WIDTH_MM}mm")
    elif cmd == 'c':
        gripper.send_command(f'G{GRIPPER_SERVO_CLOSE_ANGLE}')
        print(f"Closed to {GRIPPER_MIN_WIDTH_MM}mm")
    elif cmd == 'test':
        print("Running test sequence...")
        for w in test_values:
            angle = gripper.mm_to_angle(w)
            gripper.send_command(f'G{angle}')
            measured = input(f"   {w}mm ({angle}°) → วัดได้จริง (mm): ")
            print(f"   Expected: {w}mm, Measured: {measured}mm")
            time.sleep(0.5)
    else:
        try:
            width = float(cmd)
            angle = gripper.mm_to_angle(width)
            gripper.send_command(f'G{angle}')
            print(f"Set to {width}mm ({angle}°) - วัดจริงดูว่าตรงไหม")
        except:
            print("❌ Invalid")

---
# 🎯 MAIN OPERATION
---

## 📷 Capture Background + Floor Depth

In [ ]:
robot.home()

In [8]:
print("="*50)
print("📷 BACKGROUND + FLOOR CALIBRATION")
print("⚠️ เอาวัตถุออกให้หมด!")
print("SPACE = Capture | Q = Quit")
print("="*50)

cap = cv2.VideoCapture(CAMERA_ID)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    cv2.putText(frame, "REMOVE ALL OBJECTS - Press SPACE", 
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow('Calibration', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):
        detector.set_background(frame)
        depth_estimator.calibrate_floor(frame)
        print("✅ Ready!")
        break
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

📷 BACKGROUND + FLOOR CALIBRATION
⚠️ เอาวัตถุออกให้หมด!
SPACE = Capture | Q = Quit
✅ Background captured!
✅ Floor depth: 3.3829
✅ Ready!


In [ ]:
robot.home()

## 🎯 Pick-and-Place

In [9]:
# =================================================================
# 🎯 PICK-AND-PLACE v5.1
# =================================================================
selected_object = None
detected_objects = []
current_depth_map = None

def mouse_callback(event, x, y, flags, param):
    global selected_object
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in detected_objects:
            bx, by, bw, bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                grip_width, grip_angle, robot_r = detector.get_grip_params(obj, PIXELS_PER_MM)
                cx, cy = obj['center']
                robot_x, robot_y = robot.pixel_to_robot(cx, cy)
                height_mm = depth_estimator.get_object_height_mm(current_depth_map, obj, DEPTH_Z_SCALE) if current_depth_map is not None else 0
                z_grasp = depth_estimator.calculate_grasp_z(height_mm)
                
                print(f"\n📦 Selected: W={grip_width:.1f}mm R={robot_r:.1f}° Z={z_grasp:.1f}")
                break

def draw_grip(frame, obj, color):
    rect = obj['rect']
    box = cv2.boxPoints(rect)
    cv2.drawContours(frame, [np.int32(box)], 0, color, 2)
    cx, cy = obj['center']
    _, grip_angle, _ = detector.get_grip_params(obj, PIXELS_PER_MM)
    dx = int(40 * np.cos(np.radians(grip_angle)))
    dy = int(40 * np.sin(np.radians(grip_angle)))
    cv2.line(frame, (cx-dx, cy-dy), (cx+dx, cy+dy), (0, 0, 255), 3)

def pick_object(obj):
    cx, cy = obj['center']
    grip_width, _, robot_r = detector.get_grip_params(obj, PIXELS_PER_MM)
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    height_mm = depth_estimator.get_object_height_mm(current_depth_map, obj, DEPTH_Z_SCALE) if current_depth_map is not None else 0
    z_grasp = depth_estimator.calculate_grasp_z(height_mm)
    
    print(f"\n🤖 Picking: W={grip_width:.1f}mm R={robot_r:.1f}° Z={z_grasp:.1f}")
    
    # 🔄 Move to safe joint position first to avoid collision
    print("🔄 Moving to safe joint position (0,0,0,0)...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    gripper.open_for_object(grip_width)
    time.sleep(1)
    robot.move_to_and_wait(robot_x, robot_y, Z_APPROACH, robot_r, 3)
    robot.move_to_and_wait(robot_x, robot_y, z_grasp, robot_r, 2)
    gripper.grip_object(grip_width)
    time.sleep(1.5)
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, robot_r, 2)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    gripper.release()
    time.sleep(1)
    robot.home()
    print("✅ Complete!")

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v5.1')
cv2.setMouseCallback('Pick v5.1', mouse_callback)

show_depth = False
frame_count = 0

print("Click=Select | SPACE=Pick | D=Depth | H=Home | Q=Quit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame_count += 1
    if frame_count % 10 == 0:
        current_depth_map = depth_estimator.estimate_depth(frame)
    
    detected_objects = detector.detect_objects(frame, THRESHOLD_VALUE, MIN_OBJECT_AREA)
    
    for obj in detected_objects:
        x, y, w, h = obj['bbox']
        is_selected = selected_object and obj['center'] == selected_object['center']
        color = (0, 0, 255) if is_selected else (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        draw_grip(frame, obj, color)
        grip_width, _, robot_r = detector.get_grip_params(obj, PIXELS_PER_MM)
        cv2.putText(frame, f"W:{grip_width:.0f} R:{robot_r:.0f}", (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    cv2.rectangle(frame, (0, 0), (640, 35), (50, 50, 50), -1)
    cv2.putText(frame, f"Objects: {len(detected_objects)} | Click | SPACE=Pick | D=Depth | Q=Quit",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    if selected_object:
        cv2.putText(frame, "[SELECTED - SPACE to pick]", (10, 470),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
    if show_depth and current_depth_map is not None:
        d = (current_depth_map - current_depth_map.min()) / (current_depth_map.max() - current_depth_map.min())
        d = cv2.applyColorMap((d * 255).astype(np.uint8), cv2.COLORMAP_INFERNO)
        d = cv2.resize(d, (frame.shape[1], frame.shape[0]))
        frame = cv2.addWeighted(frame, 0.6, d, 0.4, 0)
    
    cv2.imshow('Pick v5.1', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'): selected_object = None
    elif key == ord('h'): robot.home()
    elif key == ord('d'): show_depth = not show_depth
    elif key == ord(' ') and selected_object:
        pick_object(selected_object)
        selected_object = None

cap.release()
cv2.destroyAllWindows()

Click=Select | SPACE=Pick | D=Depth | H=Home | Q=Quit

📦 Selected: W=38.3mm R=-90.0° Z=-61.8

🤖 Picking: W=38.3mm R=-90.0° Z=-63.7
🔄 Moving to safe joint position (0,0,0,0)...
   → JointMovJ(0,0,0,0)
🦾 Opening for 38.3mm → 53.3mm (25°)
   → MovJ(0.5358603000640869,5.878016948699951,-55,-90.0)
   → MovJ(0.5358603000640869,5.878016948699951,-63.7032356262207,-90.0)
🦾 Gripping 38.3mm → 35.3mm (59°)
   → MovJ(0.5358603000640869,5.878016948699951,-40,-90.0)
   → MovJ(-253.07,115.17,-17.07,-62.78)
🦾 Releasing → 48.3mm (39°)
🤖 Moving to HOME...
✅ HOME
✅ Complete!

📦 Selected: W=37.0mm R=-62.5° Z=-64.0

🤖 Picking: W=37.0mm R=-62.5° Z=-64.0
🔄 Moving to safe joint position (0,0,0,0)...
   → JointMovJ(0,0,0,0)
🦾 Opening for 37.0mm → 52.0mm (30°)
   → MovJ(0.8880894780158997,5.359443187713623,-55,-62.52556610107422)
   → MovJ(0.8880894780158997,5.359443187713623,-64,-62.52556610107422)
🦾 Gripping 37.0mm → 34.0mm (61°)
   → MovJ(0.8880894780158997,5.359443187713623,-40,-62.52556610107422)
   → MovJ

In [ ]:
gripper.disconnect()
print("✅ Done")